# Capstone Project - The Battle of the Neighborhoods (Week 2)


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction/ Business Problem <a name="introduction"></a>

In the past ten years Budapest has started to become a must go destination for every traveller visiting Europe. It has also become a great place to settle down or live for a short period of time, attracting many students and young professionals every year. It is a beautiful city, offering all the perks of Western capitals but at a much lower price. 

In this period, the city has seen a spike in the number of cafes offering specialty coffee, pleasant atmosphere and outstanding customer services. However, they are all very similar.  There are very few places where you could actually go drink a coffee and choose a book to read, and even less where you can buy the books that you started reading while sipping your coffee. In other words,  a place where you could go and look for interesting readings while relaxing on sofas, having your favourite drink. I think this combination will attract a lot of customers interested in reading or only in coffee, but who would like to have their drink in a new, different atmosphere.

In this project I would like to solve the following problem:   <b> Which area in Budapest would be the most suitable to open a Bookstore&Café place.</b> 
The main criteria for the selection of the location would be:
1.	Area close to university/college 
2.	Area with less competitors (bookstores and cafes)
3.	Close to central area

I have chosen these particular criteria since the target group will most probably be students/professionals/digital nomads/freelancers interested in reading or looking for a quiet, nice place to have coffee and study/read/work. Moreover, the lack of competition will definitely be an advantage. By being the only venue of such type in the area, it will make us the best by default, plus it will assure a constant flow of customers.


## Data <a name="data"></a>

To solve this particular business problem, I will use the Foursquare location data. 

I will extract/generate the following information using Foursquare APIs:
* the universities, coffee shops and bookstores located in the central area of Budapest
* determine possible 'candidate' areas (universities)
* explore the venues around those 'candidate' universities
* find the best location given the number of coffee shops and bookstores in their proximity.

## Analysis <a name="data"></a>

I have imported all necessary libraries 

In [1]:
pip install geopy

     |████████████████████████████████| 102kB 19.2MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests 
import pandas as pd
import folium
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

 I have defined Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'S4KPCJKZIODIUMGINZHTYKMOHV2I2C0N1PWL4BRXKPCZGWE5' 
CLIENT_SECRET = 'MTFOEKLBB2GMG1HX0BGC2COQJJOXOLCJDCLYHJPLZ0VBZSOU' 
VERSION = '20180604'
LIMIT = 5000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S4KPCJKZIODIUMGINZHTYKMOHV2I2C0N1PWL4BRXKPCZGWE5
CLIENT_SECRET:MTFOEKLBB2GMG1HX0BGC2COQJJOXOLCJDCLYHJPLZ0VBZSOU


I have used geopy library to get the latitude and longitude values of Budapest City

In [4]:
address = 'Budapest, HU'
geolocator = Nominatim(user_agent="budapest_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Budapest are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Budapest are 47.4983815, 19.0404707.


I have defined a function which extracts the category of the venues. I will use in in the next steps of the analysis.

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

As mentioned in the Introduction, for the selection of the best district/location, I will take in consideration three criteria: 
1. the area has to be near a university; 
2. the area should have less coffee shops and bookstores; 
3. the area should be close to the city center. 

For this I have created search queries to find universities, coffee shops and bookstores in the central area of Budapest

In [6]:
# defining the search query, the URL and sending the GET request to get the desired results 
search_query ='university'
radius=3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
results = requests.get(url).json()
venues = results['response']['venues']

# tranforming venues into a dataframe
df = json_normalize(venues)

# defining information of interest and filtering the dataframe
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_filtered




,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,neighborhood,id
0,Central European University (CEU),University,Nádor u. 9.,Zrínyi u.,47.501104,19.049355,"[{'label': 'display', 'lat': 47.50110388183741...",733,1051,HU,Budapest,Magyarország,"[Budapest, Nádor u. 9. (Zrínyi u.), 1051, Magy...",NaN,NaN,4b5605a4f964a5206cfc27e3
1,ELTE Egyetemi Könyvtár és Levéltár / ELTE Univ...,College Library,Ferenciek tere 6.,NaN,47.492537,19.057242,"[{'label': 'display', 'lat': 47.49253702166014...",1419,1053,HU,Budapest,Magyarország,"[Budapest, Ferenciek tere 6., 1053, Magyarország]",Budapest,NaN,4d4b580f2220b1f7e69a29d2
2,AIESEC Corvinus University of Budapest,Office,Fővám tér 8,NaN,47.486366,19.057116,"[{'label': 'display', 'lat': 47.48636630733371...",1832,1093,HU,Budapest,Magyarország,"[Budapest, Fővám tér 8, 1093, Magyarország]",Budapest,NaN,4d304d6b05585941e6e85b37
3,Central European University Library,College Library,Nádor utca 15,Zrínyi utca,47.501188,19.049612,"[{'label': 'display', 'lat': 47.50118807096102...",755,1051,HU,Budapest,Magyarország,"[Budapest, Nádor utca 15 (Zrínyi utca), 1051, ...",Budapest,Lipótváros,4d8793ce5ad3a093ec880efe
4,French University,University,NaN,NaN,47.497334,19.041196,"[{'label': 'display', 'lat': 47.49733449193851...",128,NaN,HU,NaN,Magyarország,[Magyarország],NaN,NaN,512373f2e4b01f5bb0307a62
5,Leo Burnett University,College Communications Building,NaN,NaN,47.496895,19.049784,"[{'label': 'display', 'lat': 47.49689483642578...",719,NaN,HU,NaN,Magyarország,[Magyarország],NaN,NaN,4f7c1b9be4b066eb233ab1e0
6,Oxford University Press,Bookstore,Gerlóczy u 7,NaN,47.495352,19.056549,"[{'label': 'display', 'lat': 47.49535236725313...",1255,1052,HU,Budapest,Magyarország,"[Budapest, Gerlóczy u 7, 1052, Magyarország]",Budapest,NaN,4ddb73098877caae8015ebcb
7,Molecular and Cellular Neuroendocrine Laborato...,College Science Building,Tűzoltó utca 58.,NaN,47.481408,19.079497,"[{'label': 'display', 'lat': 47.48140813709447...",3491,1094,HU,Budapest,Magyarország,"[Budapest, Tűzoltó utca 58., 1094, Magyarország]",Budapest,NaN,5033480d3950111ac590c453
8,Budapest University Of Technology And Economics,University,NaN,NaN,47.500313,19.024412,"[{'label': 'display', 'lat': 47.500313, 'lng':...",1226,NaN,HU,NaN,Magyarország,[Magyarország],NaN,NaN,57d68c91498e0a64130cb406
9,Eötvös Loránd University,University,Egyetem Tér,Szerb Utca,47.490769,19.058396,"[{'label': 'display', 'lat': 47.490769, 'lng':...",1592,1053,HU,Budapest,Magyarország,"[Budapest, Egyetem Tér (Szerb Utca), 1053, Mag...",Budapest,NaN,58530cecfad9dc0635aeac34


In [7]:
df_filtered.shape

(30, 16)

In [8]:
# defining the search query, the URL and sending the GET request to get the desired results 
search_query_1 ='bookstore'
url_1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_1, radius, LIMIT)
results_1 = requests.get(url_1).json()
venues_1= results_1['response']['venues']

# tranforming venues into a dataframe
df_1 = json_normalize(venues_1)

# defining information of interest and filtering the dataframe
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered_1 = df_1.loc[:, filtered_columns]
df_filtered_1['categories'] = df_filtered_1.apply(get_category_type, axis=1)
df_filtered_1.columns = [column.split('.')[-1] for column in df_filtered_1.columns]

df_filtered_1.head(2)



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,neighborhood,id
0,Libri Bookstore,Bookstore,Alkotas utca 53.,Csörsz utca,47.490705,19.022848,"[{'label': 'display', 'lat': 47.49070508878965...",1577,1124,HU,Budapest,Magyarország,"[Budapest, Alkotas utca 53. (Csörsz utca), 112...",Budapest,NaN,5ad4a33ab1ec135abfc7ed31
1,Treehugger Dan's Bookstore & Cafe,Bookstore,Csengery,NaN,47.502850,19.057764,"[{'label': 'display', 'lat': 47.50284992378462...",1392,NaN,HU,NaN,Magyarország,"[Csengery, Magyarország]",NaN,NaN,5332d0fc498e87f6a42c5fda


In [9]:
df_filtered_1.shape

(5, 16)

In [10]:
# defining the search query, the URL and sending the GET request to get the desired results 
search_query_2='coffee shop'
url_2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_2)
result_2= requests.get(url_2).json()
venues_2 = result_2['response']['venues']

# tranforming venues into a dataframe
df_2= json_normalize(venues_2)

# defining information of interest and filtering the dataframe
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered_2 = df_2.loc[:, filtered_columns]
df_filtered_2['categories'] = df_filtered_2.apply(get_category_type, axis=1)
df_filtered_2.columns = [column.split('.')[-1] for column in df_filtered_2.columns]

df_filtered_2.head(2)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,neighborhood,id
0,The Coffee Shop,Coffee Shop,Váci út. 18.,NaN,47.513242,19.057555,"[{'label': 'display', 'lat': 47.513242, 'lng':...",2094,1132,HU,Budapest XIII. kerület,Magyarország,"[Budapest XIII. kerület, Váci út. 18., 1132, M...",Budapest,NaN,5bbc4a8de96d0c002cdb74d9
1,Coffee shop company Váci Utca,Café,Váci utca 25,NaN,47.493869,19.053410,"[{'label': 'display', 'lat': 47.49386929353444...",1095,1052,HU,Budapest,Magyarország,"[Budapest, Váci utca 25, 1052, Magyarország]",Budapest,NaN,51cc461a498e564e4b85f6ac


In [11]:
df_filtered_2.shape

(30, 16)

I have used the folium libray to vizualize all the venues on the map

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add the universities as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the bookstore as red circle markers
for lat, lng, label in zip(df_filtered_1.lat, df_filtered_1.lng, df_filtered_1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the coffee shops as yellow circle markers
for lat, lng, label in zip(df_filtered_2.lat, df_filtered_2.lng, df_filtered_2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
    

# display map
venues_map

We can see that there is a lot of potential, especially in the inner city, the most central part of Budapest.

However, things don't seem quite right, so, I will do a further analysis within 500 m of the 'candidate' universities and double-check if the previous analysis has rendered all possible results. At the same time, with this analysis, I will compare the possible locations and choose the best option.

In [13]:
# coordinates of Central European University
latitude =47.501104
longitude = 19.049355
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) 


filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(2)


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,neighborhood,crossStreet,id
0,Four Seasons Hotel Gresham Palace Budapest,Hotel,Széchenyi István tér 5-6,47.499886,19.048025,"[{'label': 'display', 'lat': 47.49988569809184...",168,1051,HU,Budapest,Magyarország,"[Budapest, Széchenyi István tér 5-6, 1051, Mag...",NaN,NaN,NaN,4ba0b834f964a5204c7937e3
1,Espresso Embassy,Coffee Shop,Arany János u. 15.,47.502048,19.050175,"[{'label': 'display', 'lat': 47.502048, 'lng':...",121,1051,HU,Budapest,Magyarország,"[Budapest, Arany János u. 15., 1051, Magyarors...",Budapest,NaN,NaN,506ee78fe4b0f919387f5d1b


In [14]:
dataframe_filtered['categories'].value_counts().to_frame(name='Count')

,Count
Hotel,9
Italian Restaurant,8
Restaurant,8
Coffee Shop,6
Hungarian Restaurant,4
Wine Bar,3
Ice Cream Shop,3
Park,3
Plaza,3
Eastern European Restaurant,3


In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# add a red circle marker to represent Central European University
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='CEU',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shopps as yellow circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered[dataframe_filtered['categories'] == 'Coffee Shop'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the bookstore as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered[dataframe_filtered['categories'] == 'Bookstore'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

So, this area is a bit crowded. I will examine now the two other possibilities

In [16]:

# coordinates of Eötvös Loránd University
latitude_1 =47.490769
longitude_1 = 19.058396
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_1, longitude_1, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) 


filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_1 = dataframe.loc[:, filtered_columns]
dataframe_filtered_1['venue.categories'] = dataframe_filtered_1.apply(get_category_type, axis=1)
dataframe_filtered_1.columns = [col.split('.')[-1] for col in dataframe_filtered_1.columns]

dataframe_filtered_1.head(2)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,GoodSpirit Whisky & Coctail Bar,Cocktail Bar,Veres Pálné u. 7.,47.490926,19.056088,"[{'label': 'display', 'lat': 47.49092620880407...",174,1056,HU,Budapest,Budapest,Magyarország,"[Budapest, Veres Pálné u. 7., 1056, Magyarország]",NaN,NaN,58827c3054073d0c8ddf198d
1,Károlyi-kert,Park,Ferenczy István u.,47.492098,19.059606,"[{'label': 'display', 'lat': 47.49209844664687...",173,1053,HU,Budapest,Budapest,Magyarország,"[Budapest, Ferenczy István u. (Magyar u.), 105...",Magyar u.,NaN,4bd6c53ccfa7b7135cff27da


In [17]:
dataframe_filtered_1['categories'].value_counts().to_frame(name='Count')

,Count
Coffee Shop,10
Hungarian Restaurant,4
Vegetarian / Vegan Restaurant,4
Bar,4
Italian Restaurant,4
Restaurant,4
Hotel,4
Tea Room,3
Dessert Shop,3
Bakery,3


Here we see even more Coffee Shops but no Bookstore.

In [18]:

# coordinates of Corvinus University
latitude_2= 47.486366 
longitude_2 = 19.057116
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_2, longitude_2, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) 


filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_2 = dataframe.loc[:, filtered_columns]
dataframe_filtered_2['venue.categories'] = dataframe_filtered_2.apply(get_category_type, axis=1)
dataframe_filtered_2.columns = [col.split('.')[-1] for col in dataframe_filtered_2.columns]

dataframe_filtered_2.head(2)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Ázsia Bolt,Gourmet Shop,Vámház krt. 5.,Pipa u.,47.487522,19.058594,"[{'label': 'display', 'lat': 47.48752167006327...",170,1093,HU,Budapest,Budapest,Magyarország,"[Budapest, Vámház krt. 5. (Pipa u.), 1093, Mag...",NaN,4e578ddf7d8bf55c17293f3f
1,Pampas Steakhouse,Steakhouse,Vámház krt. 6.,Veres Pálné u.,47.487818,19.058698,"[{'label': 'display', 'lat': 47.487818, 'lng':...",200,1056,HU,Budapest,Budapest,Magyarország,"[Budapest, Vámház krt. 6. (Veres Pálné u.), 10...",NaN,4bf42104ff90c9b669df5428


In [19]:
dataframe_filtered_2['categories'].value_counts().to_frame(name='Count')

,Count
Hungarian Restaurant,6
Coffee Shop,5
Bar,5
Ice Cream Shop,4
Café,4
Plaza,4
Bakery,3
Hotel,3
Vegetarian / Vegan Restaurant,3
Italian Restaurant,3


In [20]:
venues_map = folium.Map(location=[latitude_2, longitude_2], zoom_start=13) 

# add a red circle marker to represent Central European University
folium.features.CircleMarker(
    [latitude_2, longitude_2],
    radius=10,
    color='green',
    popup='CEU',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shopps as yellow circle markers
for lat, lng, label in zip(dataframe_filtered_2.lat, dataframe_filtered_2.lng, dataframe_filtered_2[dataframe_filtered_2['categories'] == 'Coffee Shop'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the bookstore as red circle markers
for lat, lng, label in zip(dataframe_filtered_2.lat, dataframe_filtered_2.lng, dataframe_filtered_2[dataframe_filtered_2['categories'] == 'Bookstore'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

This area is the best option so far. It has less competitors as well as very central location with a nice view over the Danube. Moreover, it is definitely a tourist hub. 

Still, I would like to consider a last option: Semmelweis University.   

In [21]:

# coordinates of Semmeslweis University
latitude_3= 47.482094 
longitude_3 = 19.081897
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_3, longitude_3, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items) 


filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_3 = dataframe.loc[:, filtered_columns]
dataframe_filtered_3['venue.categories'] = dataframe_filtered_3.apply(get_category_type, axis=1)
dataframe_filtered_3.columns = [col.split('.')[-1] for col in dataframe_filtered_3.columns]

dataframe_filtered_3.head(2)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Pesti Burger és Bár,Burger Joint,Tűzoltó utca 50-56.,47.481607,19.078767,"[{'label': 'display', 'lat': 47.481607, 'lng':...",241,1096,HU,Budapest,Budapest,Magyarország,"[Budapest, Tűzoltó utca 50-56., 1096, Magyaror...",NaN,NaN,5b06edbc840fc2002c3601b8
1,Magyar Természettudományi Múzeum,Science Museum,Ludovika tér 2-6.,47.482512,19.085218,"[{'label': 'display', 'lat': 47.48251244515745...",254,1083,HU,Budapest,Budapest,Magyarország,"[Budapest, Ludovika tér 2-6., 1083, Magyarország]",NaN,NaN,4c5da69ad25320a13a96c67a


In [22]:
dataframe_filtered_3['categories'].value_counts().to_frame(name='Count')

,Count
Park,3
Chinese Restaurant,2
Bar,2
Hotel,2
Pub,2
Food & Drink Shop,2
Burger Joint,2
Flower Shop,2
Diner,2
Bakery,1


In [23]:
venues_map = folium.Map(location=[latitude_3, longitude_3], zoom_start=13) 

# add a red circle marker to represent Semmelweis University
folium.features.CircleMarker(
    [latitude_3, longitude_3],
    radius=10,
    color='green',
    popup='Semmelweis',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as yellow circle markers
for lat, lng, label in zip(dataframe_filtered_3.lat, dataframe_filtered_3.lng, dataframe_filtered_3[dataframe_filtered_3['categories'] == 'Coffee Shop'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the bookstore as red circle markers
for lat, lng, label in zip(dataframe_filtered_2.lat, dataframe_filtered_3.lng, dataframe_filtered_3[dataframe_filtered_3['categories'] == 'Bookstore'].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

This area has a lot of potential. There is just one venue selling specialized coffee and no bookstore. Yes, it's not a central area, neither is it popular with tourists, but it's a hub for medical students which are notorious for sleepless nights and studying hard. They will definitely be devoted customers.

Now, at the end, I would like to explore the existing venue and see its rating.

In [24]:
# ID of The Nook Coffee Shop
venue_id = '5b8633860457b7002ce4fe1c' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


As I see, the venue doesn't have the best rating, so, that's another advantage.

## Results and Discussions <a name="data"></a>

The analysis has rendered 2 possible locations which are quite different. On the one hand, you have the area around Corvinus University, which is situated in a central location, close to university and has rather few competitors; and on the other hand- the area around Semmelweis University, which doesn't fulfill the initial criteria, but which could be a great candidate given the high number of students circulating in the area, as well as almost a lack of competitors.

At this point it is up to the owner to decide what's the desired target group as well as the capital they want to invest initially. 

Given the distance from the city center, the area around Semmelweis University will have much lower rent fees as well as high probability of finding more available spaces to rent. Moreover, other factors like attractiveness of each location and socio economic dynamic in the neighborhood should also be taken in consideration.

However, both indicated locations would be suitable for opening a bookstore/cafe given the proximity to university and lower competition, and of course, the novelty of the whole concept for the specific area.

## Conclusion <a name="data"></a>

In this project I have tried to determine the best areas in Budapest for opening a bookstore/cafe. Using Foursquare location data for analysis, I have proposed 2 specific areas. However, the decision is in the hands of the stakeholders now. 